## Can a `vectorize` function call a `jit` function?

In [1]:
import numpy
from numba import vectorize, jit, njit

In [2]:
@jit
def add(a, b):
    return a + b

In [3]:
@vectorize
def add_vec(a, b):
    return add(a, b)

In [4]:
to_add = numpy.arange(100).reshape(50, 2)

In [5]:
add_vec(to_add[:, 0], to_add[:, 1])

array([  1,   5,   9,  13,  17,  21,  25,  29,  33,  37,  41,  45,  49,
        53,  57,  61,  65,  69,  73,  77,  81,  85,  89,  93,  97, 101,
       105, 109, 113, 117, 121, 125, 129, 133, 137, 141, 145, 149, 153,
       157, 161, 165, 169, 173, 177, 181, 185, 189, 193, 197])

## Can a `jit` function call a `vectorize` function?

In [6]:
@vectorize
def add(a, b):
    return a + b

@jit
def add_vec(a, b):
    return add(a, b)

In [7]:
add_vec(to_add[:, 0], to_add[:, 1])

array([  1,   5,   9,  13,  17,  21,  25,  29,  33,  37,  41,  45,  49,
        53,  57,  61,  65,  69,  73,  77,  81,  85,  89,  93,  97, 101,
       105, 109, 113, 117, 121, 125, 129, 133, 137, 141, 145, 149, 153,
       157, 161, 165, 169, 173, 177, 181, 185, 189, 193, 197])

## Can I use `isinstance` in `jit`ted functions?  

No, but there is a solution, `generated_jit`

Imagine you have a function that takes an array but wants the scalar sum of that array to operate on.  You can just use `.sum()`.  But sometimes, instead of an array getting passed in, a scalar gets passed, which will throw an error since scalars don't have a `sum` method.  

In [8]:
from numba import types, generated_jit

In [9]:
@generated_jit
def safesum(M):
    if isinstance(M, types.Array):
        return lambda M: M.sum()
    else:
        return lambda M: M

In [10]:
safesum(to_add)

4950

In [11]:
safesum(5)

5

### Note:

* `safesum` gets called with the Numba types of the argument, not their values
* it returns a _function_, not a value.  

You could use this to have different functions called depending on the type of the inputs

In [12]:
def add(a, b):
    return a + b

add_nums = jit(nopython=True)(add)
add_str = jit()(add)

In [13]:
@generated_jit
def safeadd(a, b):
    if isinstance(a, types.Opaque):
        return add_str
    else:
        return add_nums

In [14]:
safeadd('3', '4')

'34'

In [15]:
safeadd(3, 4)

7

# Tips

Set envvar `NUMBA_DISABLE_JIT=1` to disable numba compilation (for debugging)

Install the "Hide Traceback" extension if you're prototyping in a notebook.

# Errors

### Unification errors

Thanks to Graham Markhall for the idea for these examples:
http://gmarkall.github.io/tutorials/pycon-uk-2015

When Numba compiles a function just-in-time, it needs to declare the type of the output(s).  If it can't do that in a consistent way, it gets upset.

In [16]:
@jit(nopython=True)
def get_low(a, b, c):
    if c:
        return a
    else:
        return b

In [17]:
get_low(3., (3, 2), True)

TypingError: Failed at nopython (nopython frontend)
[33m[1mCan't unify return type from the following types: float64, tuple(int64 x 2)
[33m[1mReturn of: IR name '$8.2', type 'tuple(int64 x 2)', location: [37m[1m
File "<ipython-input-16-91918b07f7a3>", line 6:[0m
[34m[1mdef get_low(a, b, c):
    <source elided>
    else:
[31m[1m        return b
[0m        [32m[1m^[0m[0m[0m
[33m[1mReturn of: IR name '$4.2', type 'float64', location: [37m[1m
File "<ipython-input-16-91918b07f7a3>", line 4:[0m
[34m[1mdef get_low(a, b, c):
    <source elided>
    if c:
[31m[1m        return a
[0m        [32m[1m^[0m[0m[0m[0m
This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/dev/reference/pysupported.html
and
http://numba.pydata.org/numba-doc/dev/reference/numpysupported.html

Fo

In [18]:
get_low(4, numpy.zeros(3), True)

TypingError: Failed at nopython (nopython frontend)
[33m[1mCan't unify return type from the following types: array(float64, 1d, C), int64
[33m[1mReturn of: IR name '$8.2', type 'array(float64, 1d, C)', location: [37m[1m
File "<ipython-input-16-91918b07f7a3>", line 6:[0m
[34m[1mdef get_low(a, b, c):
    <source elided>
    else:
[31m[1m        return b
[0m        [32m[1m^[0m[0m[0m
[33m[1mReturn of: IR name '$4.2', type 'int64', location: [37m[1m
File "<ipython-input-16-91918b07f7a3>", line 4:[0m
[34m[1mdef get_low(a, b, c):
    <source elided>
    if c:
[31m[1m        return a
[0m        [32m[1m^[0m[0m[0m[0m
This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/dev/reference/pysupported.html
and
http://numba.pydata.org/numba-doc/dev/reference/numpysupported.ht

In [19]:
get_low(3, 4, True)

3

In [20]:
@jit(nopython=True)
def dont_index_a_scalar(a):
    return a[0]

In [21]:
dont_index_a_scalar(5.)

TypingError: Failed at nopython (nopython frontend)
[33m[1m[33m[1m[33m[1mInvalid usage of getitem with parameters (float64, int64)
 * parameterized[0m
[0m[37m[1m[1] During: typing of intrinsic-call at <ipython-input-20-ebd0b08b1cdb> (3)[0m
[0m[37m[1m[2] During: typing of static-get-item at <ipython-input-20-ebd0b08b1cdb> (3)[0m
[37m[1m
File "<ipython-input-20-ebd0b08b1cdb>", line 3:[0m
[34m[1mdef dont_index_a_scalar(a):
[31m[1m    return a[0]
[0m    [32m[1m^[0m[0m

This is not usually a problem with Numba itself but instead often caused by
the use of unsupported features or an issue in resolving types.

To see Python/NumPy features supported by the latest release of Numba visit:
http://numba.pydata.org/numba-doc/dev/reference/pysupported.html
and
http://numba.pydata.org/numba-doc/dev/reference/numpysupported.html

For more information about typing errors and how to debug them visit:
http://numba.pydata.org/numba-doc/latest/user/troubleshoot.html#my-code-doesn

## Globals are treated as compile-time constants by Numba

In [24]:
a = 5

In [25]:
@njit
def add_to_a(b):
    return a + b

In [26]:
add_to_a(7)

12

In [27]:
a = 12

In [28]:
add_to_a(7)

12

### Solution: Don't use globals(!)

Seriously.  Don't.  But if you must, you can force a recompile of the jitted function.

In [29]:
add_to_a.recompile()

In [30]:
add_to_a(7)

19